In [78]:
import pandas as pd
import json

# Read JSON Lines file (each line is a separate JSON object)
data = []
with open('mindset.jl', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.strip()))

df = pd.DataFrame(data)

condition = 'ptsd'
df_adhd = df[df['label'].apply(lambda x: condition in x)]

data = df_adhd.explode('posts').reset_index(drop=True)

posts_normalized = pd.json_normalize(data['posts'])
keep_cols = [col for col in data.columns if col != 'posts']
data = pd.concat([data[keep_cols].reset_index(drop=True), posts_normalized], axis=1)
data.head()

,username,label,created_at,subreddit,id,title,score,selftext
0,124890,"[bipolar, ocd, anxiety, autism, ptsd, depressi...",1661153709,GeneralHospital,wun2j9,now that spencer admitted his love for trina..,2,how long do you all think it ll be before the ...
1,124890,"[bipolar, ocd, anxiety, autism, ptsd, depressi...",1658631113,SkincareAddiction,w6l9l8,can i use good molecules discoloration correct...,4,i have been dealing with hormonal acne due to ...
2,124890,"[bipolar, ocd, anxiety, autism, ptsd, depressi...",1654939984,DipPowderNails,v9u0ul,is white dip usually hard to work with or was ...,1,"i dipped my nails for the first time today, an..."
3,124890,"[bipolar, ocd, anxiety, autism, ptsd, depressi...",1654582077,scientology,v6o2yt,South Park Scientology episode End Credits,38,i noticed that they used the same two names fo...
4,124890,"[bipolar, ocd, anxiety, autism, ptsd, depressi...",1653978258,SkincareAddiction,v1jusy,[Skin Concerns] what to do/use for breakouts d...,3,i have always had clear but dry skin. but ever...


In [82]:
with open('ptsd_diagnosed_posts_map.json') as f:
  map = json.load(f)

diagnosis_data = []
for user_id, posts in map.items():
    if posts:
        # Assuming the first post in the list is the relevant diagnosis post
        diagnosis_post = posts[0]
        diagnosis_data.append({
            'user_id': int(user_id),
            'diagnosis_timestamp': diagnosis_post['created_utc']
        })

diagnosis_df = pd.DataFrame(diagnosis_data)
diagnosis_df.head()

,user_id,diagnosis_timestamp
0,236306,1643811026
1,547022,1596086102
2,220717,1715904688
3,544079,1624955284
4,161818,1688122939


In [81]:
matched_rows = data[data['diagnosis_timestamp'].notna()]
print(matched_rows[['username', 'diagnosis_timestamp']].head())

KeyError: 'diagnosis_timestamp'

In [ ]:
df_adhd_flat

,username,label,created_at,subreddit,id,title,score,selftext,user_id,diagnosis_timestamp
0,124890,"[bipolar, ocd, anxiety, autism, ptsd, depressi...",1661153709,GeneralHospital,wun2j9,now that spencer admitted his love for trina..,2,how long do you all think it ll be before the ...,NaN,NaN
1,124890,"[bipolar, ocd, anxiety, autism, ptsd, depressi...",1658631113,SkincareAddiction,w6l9l8,can i use good molecules discoloration correct...,4,i have been dealing with hormonal acne due to ...,NaN,NaN
2,124890,"[bipolar, ocd, anxiety, autism, ptsd, depressi...",1654939984,DipPowderNails,v9u0ul,is white dip usually hard to work with or was ...,1,"i dipped my nails for the first time today, an...",NaN,NaN
3,124890,"[bipolar, ocd, anxiety, autism, ptsd, depressi...",1654582077,scientology,v6o2yt,South Park Scientology episode End Credits,38,i noticed that they used the same two names fo...,NaN,NaN
4,124890,"[bipolar, ocd, anxiety, autism, ptsd, depressi...",1653978258,SkincareAddiction,v1jusy,[Skin Concerns] what to do/use for breakouts d...,3,i have always had clear but dry skin. but ever...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
507894,365951,[ptsd],1592070713,WeightLossAdvice,fupzhnz,NaN,3,yes it does- these apps also overestimate your...,365951.0,1.601424e+09
507895,365951,[ptsd],1592070121,WeightLossAdvice,fupyfkl,NaN,2,it may be more challenging for you to lose wei...,365951.0,1.601424e+09
507896,365951,[ptsd],1592066580,Celiac,fuprtvk,NaN,2,"i personally avoid grain alcohol, as i ve defi...",365951.0,1.601424e+09
507897,365951,[ptsd],1592000869,1200isfineIGUESSugh,funa7wk,NaN,2,"i got on lose it, set it for my maintenance ca...",365951.0,1.601424e+09


In [84]:
# Merge the dataframes
merged_df = pd.merge(diagnosis_df, data, on='username')

# Calculate the time difference in seconds
merged_df['time_difference_seconds'] = merged_df['diagnosis_timestamp'] - merged_df['first_post_timestamp']

# Convert to days
merged_df['time_difference_days'] = merged_df['time_difference_seconds'] / 86400

merged_df.head()

KeyError: 'username'